# CS4225/CS5425 Assignment 2


## 1. Data Preparation

In [0]:
from pyspark.sql.functions import when, col, count, isnan, mean
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum as spark_sum
import csv

In [0]:
dbutils.fs.rm("dbfs:/FileStore/tables/small_df_public_150.parquet", recurse=True)

True

In [0]:
public_150_file_path = "dbfs:/FileStore/tables/public_150k_plus_230930.csv"
df_public_150 = spark.read.csv(public_150_file_path, header=True, inferSchema=True)
df_public_150.show()

+----------+------------+-------------+----------------+--------------------+--------------------+----------------+-------------+-----------+--------------+------------+----+---------------------+---------------------+---------------------+-----------------+-------------+-------------------------+--------------------+----------------------+-------------------+--------------------+------------------+-------------------+----------------+------------+----------------------+----------------+-----------------+------------+----------+-----+------------+---------+--------------------+--------------------+-----------------+---------------+-------------------------+------------+----------------------+-------------------+---------------------+--------------------+---------------------------+--------------------+---------------------+----------------------+------------+-----------+---------+-----------------+---------------+
|LoanNumber|DateApproved|SBAOfficeCode|ProcessingMethod|        Borrower

In [0]:
# def custom_parser(line):
#     return next(csv.reader([line], delimiter=','))

# rdd = spark.sparkContext.textFile("dbfs:/FileStore/tables/public_150k_plus_230930.csv")
# header = rdd.first()
# data_rdd = rdd.filter(lambda line: line != header)

# parsed_rdd = data_rdd.map(custom_parser)
# df_public_150 = parsed_rdd.toDF(header.split(','))

In [0]:
display(df_public_150.select("LoanStatus").distinct())

df2 = df_public_150.groupBy("LoanStatus").count()
df2.show()

LoanStatus
Charged Off
Exemption 4
Paid in Full
08/19/2021
01/13/2022
02/18/2021


+------------+------+
|  LoanStatus| count|
+------------+------+
| Charged Off| 15879|
| Exemption 4| 15093|
|Paid in Full|937550|
|  08/19/2021|     1|
|  01/13/2022|     1|
|  02/18/2021|     1|
+------------+------+



In [0]:
small_df_public_150 = df_public_150.sampleBy("LoanStatus", fractions={"Charged Off": 1, "Exemption 4": 1, "Paid in Full": (15093+15879)/937553}, seed=42)
df2 = small_df_public_150.groupBy("LoanStatus").count()
df2.show()

+------------+-----+
|  LoanStatus|count|
+------------+-----+
| Charged Off|15879|
| Exemption 4|15093|
|Paid in Full|31221|
+------------+-----+



In [0]:
small_df_public_150.cache()

DataFrame[LoanNumber: bigint, DateApproved: date, SBAOfficeCode: int, ProcessingMethod: string, BorrowerName: string, BorrowerAddress: string, BorrowerCity: string, BorrowerState: string, BorrowerZip: string, LoanStatusDate: string, LoanStatus: string, Term: string, SBAGuarantyPercentage: int, InitialApprovalAmount: double, CurrentApprovalAmount: double, UndisbursedAmount: double, FranchiseName: string, ServicingLenderLocationID: int, ServicingLenderName: string, ServicingLenderAddress: string, ServicingLenderCity: string, ServicingLenderState: string, ServicingLenderZip: string, RuralUrbanIndicator: string, HubzoneIndicator: string, LMIIndicator: string, BusinessAgeDescription: string, ProjectCity: string, ProjectCountyName: string, ProjectState: string, ProjectZip: string, CD: string, JobsReported: string, NAICSCode: int, Race: string, Ethnicity: string, UTILITIES_PROCEED: string, PAYROLL_PROCEED: double, MORTGAGE_INTEREST_PROCEED: double, RENT_PROCEED: double, REFINANCE_EIDL_PROCEED

In [0]:
mapping_dict = {"Paid in Full": 1, "Charged Off": 0, "Exemption 4": 0}

for old_value, new_value in mapping_dict.items():
    small_df_public_150 = small_df_public_150.withColumn("LoanStatus", when(small_df_public_150["LoanStatus"] == old_value, new_value).otherwise(small_df_public_150["LoanStatus"]))

small_df_public_150.groupBy("LoanStatus").count().show()

+----------+-----+
|LoanStatus|count|
+----------+-----+
|         0|30972|
|         1|31221|
+----------+-----+



In [0]:
small_df_path = "dbfs:/FileStore/tables/small_df_public_150.parquet"

small_df_public_150.write.parquet(small_df_path)


In [0]:
from pyspark.sql.functions import when, col, count, isnan, mean
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum as spark_sum

In [0]:
small_df_public_150_path = "dbfs:/FileStore/tables/small_df_public_150.parquet"
small_df_public_150 = spark.read.parquet(small_df_public_150_path, header=True, inferSchema=True)

In [0]:
cols_not_avail_at_loan_time = ["LoanNumber","DateApproved","SBAOfficeCode","ProcessingMethod","LoanStatusDate","SBAGuarantyPercentage","CurrentApprovalAmount","UndisbursedAmount","ServicingLenderLocationID","ServicingLenderName","ServicingLenderAddress","ServicingLenderCity","ServicingLenderState","ServicingLenderZip","ForgivenessAmount","ForgivenessDate"]
small_df_public_150 = small_df_public_150.drop(*cols_not_avail_at_loan_time)

In [0]:
small_df_public_150.printSchema()

root
 |-- BorrowerName: string (nullable = true)
 |-- BorrowerAddress: string (nullable = true)
 |-- BorrowerCity: string (nullable = true)
 |-- BorrowerState: string (nullable = true)
 |-- BorrowerZip: string (nullable = true)
 |-- LoanStatus: string (nullable = true)
 |-- Term: string (nullable = true)
 |-- InitialApprovalAmount: double (nullable = true)
 |-- FranchiseName: string (nullable = true)
 |-- RuralUrbanIndicator: string (nullable = true)
 |-- HubzoneIndicator: string (nullable = true)
 |-- LMIIndicator: string (nullable = true)
 |-- BusinessAgeDescription: string (nullable = true)
 |-- ProjectCity: string (nullable = true)
 |-- ProjectCountyName: string (nullable = true)
 |-- ProjectState: string (nullable = true)
 |-- ProjectZip: string (nullable = true)
 |-- CD: string (nullable = true)
 |-- JobsReported: string (nullable = true)
 |-- NAICSCode: integer (nullable = true)
 |-- Race: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- UTILITIES_PROCEED: 

In [0]:
small_df_public_150.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in small_df_public_150.columns]).show()

+------------+---------------+------------+-------------+-----------+----------+----+---------------------+-------------+-------------------+----------------+------------+----------------------+-----------+-----------------+------------+----------+---+------------+---------+----+---------+-----------------+---------------+-------------------------+------------+----------------------+-------------------+---------------------+------------+---------------------------+-----------------+---------------------+----------------------+------+-------+---------+
|BorrowerName|BorrowerAddress|BorrowerCity|BorrowerState|BorrowerZip|LoanStatus|Term|InitialApprovalAmount|FranchiseName|RuralUrbanIndicator|HubzoneIndicator|LMIIndicator|BusinessAgeDescription|ProjectCity|ProjectCountyName|ProjectState|ProjectZip| CD|JobsReported|NAICSCode|Race|Ethnicity|UTILITIES_PROCEED|PAYROLL_PROCEED|MORTGAGE_INTEREST_PROCEED|RENT_PROCEED|REFINANCE_EIDL_PROCEED|HEALTH_CARE_PROCEED|DEBT_INTEREST_PROCEED|BusinessType|O

In [0]:
candidate_col =[ "OriginatingLenderCity", 
    "BorrowerCity",
    "Term",
    "InitialApprovalAmount",
    "RuralUrbanIndicator",
    "HubzoneIndicator",
    "LMIIndicator",
    "BusinessAgeDescription",
    "ProjectCity",
    "JobsReported",
    "PAYROLL_PROCEED",
    "BusinessType",
    "LoanStatus",
    "NAICSCode",
    "DEBT_INTEREST_PROCEED",
    "MORTGAGE_INTEREST_PROCEED",
    "REFINANCE_EIDL_PROCEED",
    "Gender", 
    "Veteran",
]
features_df = small_df_public_150.select(candidate_col)

features_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in features_df.columns]).show()

+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|OriginatingLenderCity|BorrowerCity|Term|InitialApprovalAmount|RuralUrbanIndicator|HubzoneIndicator|LMIIndicator|BusinessAgeDescription|ProjectCity|JobsReported|PAYROLL_PROCEED|BusinessType|LoanStatus|NAICSCode|DEBT_INTEREST_PROCEED|MORTGAGE_INTEREST_PROCEED|REFINANCE_EIDL_PROCEED|Gender|Veteran|
+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|                    0|           0|   0|                    0|                  0|               0|      

In [0]:
features_df.printSchema()

features_df = features_df.withColumn("Term", features_df["Term"].cast(IntegerType()))
features_df = features_df.withColumn("InitialApprovalAmount", features_df["InitialApprovalAmount"].cast("double"))
features_df = features_df.withColumn("JobsReported", features_df["JobsReported"].cast(IntegerType()))
features_df = features_df.withColumn("PAYROLL_PROCEED", features_df["PAYROLL_PROCEED"].cast("double"))
features_df = features_df.withColumn("DEBT_INTEREST_PROCEED", features_df["DEBT_INTEREST_PROCEED"].cast("double"))
features_df = features_df.withColumn("REFINANCE_EIDL_PROCEED", features_df["REFINANCE_EIDL_PROCEED"].cast("double"))


features_df.cache()



root
 |-- OriginatingLenderCity: string (nullable = true)
 |-- BorrowerCity: string (nullable = true)
 |-- Term: string (nullable = true)
 |-- InitialApprovalAmount: double (nullable = true)
 |-- RuralUrbanIndicator: string (nullable = true)
 |-- HubzoneIndicator: string (nullable = true)
 |-- LMIIndicator: string (nullable = true)
 |-- BusinessAgeDescription: string (nullable = true)
 |-- ProjectCity: string (nullable = true)
 |-- JobsReported: string (nullable = true)
 |-- PAYROLL_PROCEED: double (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- LoanStatus: string (nullable = true)
 |-- NAICSCode: integer (nullable = true)
 |-- DEBT_INTEREST_PROCEED: double (nullable = true)
 |-- MORTGAGE_INTEREST_PROCEED: double (nullable = true)
 |-- REFINANCE_EIDL_PROCEED: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Veteran: string (nullable = true)



DataFrame[OriginatingLenderCity: string, BorrowerCity: string, Term: int, InitialApprovalAmount: double, RuralUrbanIndicator: string, HubzoneIndicator: string, LMIIndicator: string, BusinessAgeDescription: string, ProjectCity: string, JobsReported: int, PAYROLL_PROCEED: double, BusinessType: string, LoanStatus: string, NAICSCode: int, DEBT_INTEREST_PROCEED: double, MORTGAGE_INTEREST_PROCEED: double, REFINANCE_EIDL_PROCEED: double, Gender: string, Veteran: string]

In [0]:
features_df.printSchema()

features_df.groupBy("BusinessAgeDescription").count().show()

features_df.groupBy("NAICSCode").count().show()

root
 |-- OriginatingLenderCity: string (nullable = true)
 |-- BorrowerCity: string (nullable = true)
 |-- Term: integer (nullable = true)
 |-- InitialApprovalAmount: double (nullable = true)
 |-- RuralUrbanIndicator: string (nullable = true)
 |-- HubzoneIndicator: string (nullable = true)
 |-- LMIIndicator: string (nullable = true)
 |-- BusinessAgeDescription: string (nullable = true)
 |-- ProjectCity: string (nullable = true)
 |-- JobsReported: integer (nullable = true)
 |-- PAYROLL_PROCEED: double (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- LoanStatus: string (nullable = true)
 |-- NAICSCode: integer (nullable = true)
 |-- DEBT_INTEREST_PROCEED: double (nullable = true)
 |-- MORTGAGE_INTEREST_PROCEED: double (nullable = true)
 |-- REFINANCE_EIDL_PROCEED: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Veteran: string (nullable = true)

+----------------------+-----+
|BusinessAgeDescription|count|
+----------------------+-----+
|  Existing o

In [0]:
features_df.groupBy("PAYROLL_PROCEED").count().show()
display(features_df.select("JobsReported").distinct())
features_df.groupBy("JobsReported").count().show()

# Calculate correlation between 'JobsReported' and 'PAYROLL_PROCEED'
correlation = features_df.corr("JobsReported", "PAYROLL_PROCEED")

# Print the correlation
print("Correlation between JobsReported and PAYROLL_PROCEED:", correlation)

# Calculate the sum of 'payroll_proceed' and 'JobsReported' columns
sum_payroll_proceed = features_df.select(spark_sum("PAYROLL_PROCEED")).collect()[0][0]
sum_jobs_reported = features_df.select(spark_sum("JobsReported")).collect()[0][0]

# Calculate the ratio of sum(payroll_proceed) to sum(JobsReported)
pay_ratio = sum_payroll_proceed / sum_jobs_reported

# Print the result
print("Sum of Payroll Proceeds:", sum_payroll_proceed)
print("Sum of Jobs Created:", sum_jobs_reported)
print("Ratio of average pay", pay_ratio)


+---------------+-----+
|PAYROLL_PROCEED|count|
+---------------+-----+
|      213120.18|    1|
|       223575.0|    2|
|       222349.0|    1|
|       214045.0|    1|
|       209550.0|    1|
|       204098.0|    1|
|       198940.0|    1|
|       193147.0|    1|
|       191690.0|    2|
|       168365.0|    1|
|       164480.0|    1|
|       154817.0|    2|
|       153257.0|    1|
|      775128.45|    1|
|       378097.0|    1|
|       300000.0|  120|
|       217934.0|    1|
|       216988.0|    1|
|       170041.0|    1|
|      604210.92|    1|
+---------------+-----+
only showing top 20 rows



JobsReported
148
392
31
85
251
137
65
53
133
78


+------------+-----+
|JobsReported|count|
+------------+-----+
|         148|   26|
|         392|    8|
|          31|  718|
|          85|  168|
|         251|   11|
|         137|   40|
|          65|  296|
|          53|  272|
|         133|   37|
|          78|  140|
|         322|    4|
|         362|    4|
|         155|   36|
|         108|   61|
|          34|  664|
|         193|   21|
|         101|   77|
|         115|   74|
|         126|   39|
|          81|  123|
+------------+-----+
only showing top 20 rows

Correlation between JobsReported and PAYROLL_PROCEED: 0.7176804835115626
Sum of Payroll Proceeds: 32085428682.91999
Sum of Jobs Created: 3117764
Ratio of average pay 10291.16658057505


In [0]:
# Replace null values in payroll_proceed with JobsReported * pay_ratio
features_df = features_df.withColumn("PAYROLL_PROCEED", when(col("PAYROLL_PROCEED").isNull(), col("JobsReported") * pay_ratio).otherwise(col("PAYROLL_PROCEED")))

# Show DataFrame
features_df.cache()
features_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in features_df.columns]).show()

+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|OriginatingLenderCity|BorrowerCity|Term|InitialApprovalAmount|RuralUrbanIndicator|HubzoneIndicator|LMIIndicator|BusinessAgeDescription|ProjectCity|JobsReported|PAYROLL_PROCEED|BusinessType|LoanStatus|NAICSCode|DEBT_INTEREST_PROCEED|MORTGAGE_INTEREST_PROCEED|REFINANCE_EIDL_PROCEED|Gender|Veteran|
+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|                    0|           0|   0|                    0|                  0|               0|      

In [0]:
display(features_df.select("BusinessType").distinct())
features_df.groupBy("BusinessType").count().show(23)

BusinessType
501(c)6 – Non Profit Membership
Professional Association
null
Self-Employed Individuals
Limited Liability Company(LLC)
Sole Proprietorship
Joint Venture
Partnership
Non-Profit Organization
Cooperative


+--------------------+-----+
|        BusinessType|count|
+--------------------+-----+
|501(c)6 – Non Pro...|   38|
|Professional Asso...|  329|
|                NULL|   38|
|Self-Employed Ind...|  260|
|Limited  Liabilit...|20946|
| Sole Proprietorship| 1554|
|       Joint Venture|   14|
|         Partnership| 1087|
|Non-Profit Organi...| 2462|
|         Cooperative|  126|
|Employee Stock Ow...|   30|
|               Trust|   20|
|         Corporation|24984|
|501(c)3 – Non Profit|  108|
|Limited Liability...|  756|
|Subchapter S Corp...| 9337|
|Independent Contr...|   18|
|   Single Member LLC|   38|
|Non-Profit Childc...|   31|
|501(c)19 – Non Pr...|    1|
|       Housing Co-op|   12|
|     Tribal Concerns|    3|
|    Tenant in Common|    1|
+--------------------+-----+



In [0]:
features_df = features_df.withColumn("BusinessType", when(col("BusinessType").isNull(), "Others").otherwise(col("BusinessType")))
features_df.groupBy("BusinessType").count().show(23)

+--------------------+-----+
|        BusinessType|count|
+--------------------+-----+
|501(c)6 – Non Pro...|   38|
|Professional Asso...|  329|
|Self-Employed Ind...|  260|
|Limited  Liabilit...|20946|
| Sole Proprietorship| 1554|
|       Joint Venture|   14|
|         Partnership| 1087|
|Non-Profit Organi...| 2462|
|         Cooperative|  126|
|Employee Stock Ow...|   30|
|               Trust|   20|
|              Others|   38|
|         Corporation|24984|
|501(c)3 – Non Profit|  108|
|Limited Liability...|  756|
|Subchapter S Corp...| 9337|
|Independent Contr...|   18|
|   Single Member LLC|   38|
|Non-Profit Childc...|   31|
|501(c)19 – Non Pr...|    1|
|       Housing Co-op|   12|
|     Tribal Concerns|    3|
|    Tenant in Common|    1|
+--------------------+-----+



In [0]:
display(features_df.select("NAICSCode").distinct())
most_common_naics = features_df.groupBy("NAICSCode").count().orderBy(col("count").desc()).first()[0]
features_df = features_df.withColumn("NAICSCode", when(col("NAICSCode").isNull(), most_common_naics).otherwise(col("NAICSCode")))

NAICSCode
712110
722514
561910
445291
311710
451212
333921
332710
541611
454110


In [0]:
# debt_average_value = features_df.select(mean("DEBT_INTEREST_PROCEED")).collect()[0][0]
features_df = features_df.withColumn("DEBT_INTEREST_PROCEED", when(col("DEBT_INTEREST_PROCEED").isNull(), -1).otherwise(col("DEBT_INTEREST_PROCEED")))

features_df.cache()

features_df = features_df.withColumn("MORTGAGE_INTEREST_PROCEED", when(col("MORTGAGE_INTEREST_PROCEED").isNull(), -1).otherwise(col("MORTGAGE_INTEREST_PROCEED")))

features_df = features_df.withColumn("REFINANCE_EIDL_PROCEED", when(col("REFINANCE_EIDL_PROCEED").isNull(), -1).otherwise(col("REFINANCE_EIDL_PROCEED")))

features_df.cache()

DataFrame[OriginatingLenderCity: string, BorrowerCity: string, Term: int, InitialApprovalAmount: double, RuralUrbanIndicator: string, HubzoneIndicator: string, LMIIndicator: string, BusinessAgeDescription: string, ProjectCity: string, JobsReported: int, PAYROLL_PROCEED: double, BusinessType: string, LoanStatus: string, NAICSCode: int, DEBT_INTEREST_PROCEED: double, MORTGAGE_INTEREST_PROCEED: double, REFINANCE_EIDL_PROCEED: double, Gender: string, Veteran: string]

In [0]:
display(features_df.select("Term").distinct())

features_df.groupBy("RuralUrbanIndicator").count().show()
features_df.groupBy("HubzoneIndicator").count().show()
features_df.groupBy("LMIIndicator").count().show()
features_df.groupBy("ProjectCity").count().show()
features_df.groupBy("BorrowerCity").count().show()

features_df = features_df.withColumn("Term", features_df["Term"].cast(IntegerType()))

features_df.cache()

Term
31
53
34
193
81
28
76
27
26
44


+-------------------+-----+
|RuralUrbanIndicator|count|
+-------------------+-----+
|                  U|55419|
|                  R| 6774|
+-------------------+-----+

+----------------+-----+
|HubzoneIndicator|count|
+----------------+-----+
|               Y|16789|
|               N|45404|
+----------------+-----+

+------------+-----+
|LMIIndicator|count|
+------------+-----+
|           Y|16897|
|           N|45296|
+------------+-----+

+-------------+-----+
|  ProjectCity|count|
+-------------+-----+
|       NOVATO|   11|
|    SAN PABLO|    2|
|  Middlefield|    3|
|  EAST HADDAM|    1|
|       Corona|   46|
|       GILROY|    5|
|        CHINO|    9|
|  WATSONVILLE|    8|
|      MAXWELL|    1|
|      Truckee|    2|
|     STAMFORD|   21|
|       Auburn|   21|
|  studio city|    1|
|     Thornton|    5|
|   Wellington|   12|
|   PALM BEACH|    1|
|  LAKE HARBOR|    1|
|      TUJUNGA|    6|
|SAN JUAN CAPO|    3|
|      Corning|    1|
+-------------+-----+
only showing top 20 rows


DataFrame[OriginatingLenderCity: string, BorrowerCity: string, Term: int, InitialApprovalAmount: double, RuralUrbanIndicator: string, HubzoneIndicator: string, LMIIndicator: string, BusinessAgeDescription: string, ProjectCity: string, JobsReported: int, PAYROLL_PROCEED: double, BusinessType: string, LoanStatus: string, NAICSCode: int, DEBT_INTEREST_PROCEED: double, MORTGAGE_INTEREST_PROCEED: double, REFINANCE_EIDL_PROCEED: double, Gender: string, Veteran: string]

In [0]:
features_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in features_df.columns]).show()

+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|OriginatingLenderCity|BorrowerCity|Term|InitialApprovalAmount|RuralUrbanIndicator|HubzoneIndicator|LMIIndicator|BusinessAgeDescription|ProjectCity|JobsReported|PAYROLL_PROCEED|BusinessType|LoanStatus|NAICSCode|DEBT_INTEREST_PROCEED|MORTGAGE_INTEREST_PROCEED|REFINANCE_EIDL_PROCEED|Gender|Veteran|
+---------------------+------------+----+---------------------+-------------------+----------------+------------+----------------------+-----------+------------+---------------+------------+----------+---------+---------------------+-------------------------+----------------------+------+-------+
|                    0|           0|   0|                    0|                  0|               0|      

In [0]:
features_df.printSchema()

root
 |-- OriginatingLenderCity: string (nullable = true)
 |-- BorrowerCity: string (nullable = true)
 |-- Term: integer (nullable = true)
 |-- InitialApprovalAmount: double (nullable = true)
 |-- RuralUrbanIndicator: string (nullable = true)
 |-- HubzoneIndicator: string (nullable = true)
 |-- LMIIndicator: string (nullable = true)
 |-- BusinessAgeDescription: string (nullable = true)
 |-- ProjectCity: string (nullable = true)
 |-- JobsReported: integer (nullable = true)
 |-- PAYROLL_PROCEED: double (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- LoanStatus: string (nullable = true)
 |-- NAICSCode: integer (nullable = true)
 |-- DEBT_INTEREST_PROCEED: double (nullable = true)
 |-- MORTGAGE_INTEREST_PROCEED: double (nullable = true)
 |-- REFINANCE_EIDL_PROCEED: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Veteran: string (nullable = true)



In [0]:
# Split DataFrame into training and test datasets (70% training, 30% test)
training_df, test_df = features_df.randomSplit([0.7, 0.3], seed=42)

# Print the counts of training and test datasets
print("Training Dataset Count:", training_df.count())
print("Test Dataset Count:", test_df.count())

Training Dataset Count: 43808
Test Dataset Count: 18385


In [0]:
from pyspark.sql.functions import spark_partition_id
spark.conf.get("spark.sql.shuffle.partitions")
training_df.withColumn("partition_id", spark_partition_id()).groupBy("partition_id").count().show()
test_df.withColumn("partition_id", spark_partition_id()).groupBy("partition_id").count().show()


+------------+-----+
|partition_id|count|
+------------+-----+
|           0| 6585|
|           1| 6150|
|           2| 6103|
|           3| 5897|
|           4| 5011|
|           5| 4856|
|           6| 4609|
|           7| 4597|
+------------+-----+

+------------+-----+
|partition_id|count|
+------------+-----+
|           0| 2703|
|           1| 2590|
|           2| 2550|
|           3| 2402|
|           4| 2172|
|           5| 2103|
|           6| 1962|
|           7| 1903|
+------------+-----+



## 2. Build ML Pipeline


In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder, PCA
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import countDistinct

In [0]:
categorical_cols = [ "OriginatingLenderCity", "BorrowerCity",'LMIIndicator','HubzoneIndicator',"BusinessType","ProjectCity","BusinessAgeDescription", "Gender", "Veteran"]

# label = "LoanStatus"
# features_df.select("LoanStatus").distinct().count()

In [0]:
# Dictionary to store results
distinct_counts = {}

for col in categorical_cols:
    distinct_counts[col] = features_df.select(col).distinct().count()

# Display distinct counts
for col, count in distinct_counts.items():
    print(f"Distinct values in column '{col}': {count}")

Distinct values in column 'OriginatingLenderCity': 2000
Distinct values in column 'BorrowerCity': 10295
Distinct values in column 'LMIIndicator': 2
Distinct values in column 'HubzoneIndicator': 2
Distinct values in column 'BusinessType': 23
Distinct values in column 'ProjectCity': 10294
Distinct values in column 'BusinessAgeDescription': 5
Distinct values in column 'Gender': 3
Distinct values in column 'Veteran': 3


In [0]:
high_cardinality_features = ["OriginatingLenderCity","BorrowerCity","BusinessType","ProjectCity"]

low_cardinality_features = ['LMIIndicator','HubzoneIndicator',"BusinessAgeDescription","Gender","Veteran"]

categorical_features = high_cardinality_features + low_cardinality_features

numerical_features = ['Term','InitialApprovalAmount','JobsReported','PAYROLL_PROCEED',"DEBT_INTEREST_PROCEED","MORTGAGE_INTEREST_PROCEED","REFINANCE_EIDL_PROCEED"]


# Create a list of indexers for high cardinality features
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep") for col in categorical_features]

# One-hot encode high cardinality features
encoder = OneHotEncoder(inputCols=[col + "_index" for col in categorical_features], outputCols=[col + "_encoded" for col in categorical_features])

# high_cardinality_assembler = VectorAssembler(inputCols=[col + "_index" for col in high_cardinality_features], outputCol="high_cardinality_features")

low_cardinality_assembler = VectorAssembler(inputCols=[col + "_encoded" for col in low_cardinality_features], outputCol="low_cardinality_features")

# Apply PCA to high cardinality features
# pca = PCA(k=3, inputCol="high_cardinality_features", outputCol="pca_features")

numerical_assembler = VectorAssembler(inputCols= numerical_features, outputCol="numerical_features")

# Scale numerical features
scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_numerical", withStd=True, withMean=True)

# Assemble PCA features and scaled numerical features into a single vector
assembler = VectorAssembler(inputCols=["scaled_numerical", "low_cardinality_features"], outputCol="features")

label_indexer = StringIndexer(inputCol="LoanStatus", outputCol="label")

# Define Random Forest Classifier
rf_classifier = RandomForestClassifier(featuresCol="features", labelCol="label")


In [0]:
# Define the pipeline

pipeline = Pipeline(stages= [label_indexer] + indexers + [encoder, numerical_assembler, scaler, low_cardinality_assembler, assembler, rf_classifier])

model = pipeline.fit(training_df)

predictions = model.transform(test_df)

In [0]:
# param_grid = ParamGridBuilder() \
#     .addGrid(rf_classifier.numTrees, [50, 100, 150]) \
#     .addGrid(rf_classifier.maxDepth, [5, 7, 9]) \
#     .build()

# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# cross_val = CrossValidator(estimator=pipeline,
#                            estimatorParamMaps=param_grid,
#                            evaluator=evaluator,
#                            numFolds=3) 
# # Train the model
# cv_model = cross_val.fit(training_df)


## 3. Evaluate ML Pipeline

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
# train_predictions = cv_model.transform(training_df)

# # Evaluate the model
# accuracy = evaluator.evaluate(train_predictions)
# print("Test Accuracy:", accuracy)

# # Best model
# best_model = cv_model.bestModel
# print("Best Model Parameters:")
# print("Num Trees:", best_model.stages[-1]._java_obj.getNumTrees())
# print("Max Depth:", best_model.stages[-1]._java_obj.getMaxDepth())


In [0]:
# Make predictions on the test dataset
train_predictions = model.transform(training_df)

# Evaluate the model on the test dataset
training_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
training_accuracy = training_evaluator.evaluate(train_predictions)
print("Test Accuracy:", training_accuracy)

Test Accuracy: 0.8136185171658145


In [0]:
# Make predictions on the test dataset
test_predictions = model.transform(test_df)

# Evaluate the model on the test dataset
test_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
test_accuracy = test_evaluator.evaluate(test_predictions)
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.8110416100081588
